In [ ]:
#!pip install sqlalchemy_utils
#!pip install kaggle
#!pip install streamlit

# 📊 Project HDB Resale Hackers: Extract Transform Load
---


---

## <font color='blue'>**Step 1** </font> - Import Python modules

| Import Statement | Purpose |
| :--- | :--- |
| `import os` | Used to talk to the computer's operating system. It lets the notebook do things like checking file paths and creating folders where the data will be saved.|
| `from kaggle.api.kaggle_api_extended import KaggleApi` | Imports the specific tool needed to connect to Kaggle. This object handles authentication and allows us to run commands (like downloading datasets) directly from the notebook. |
| `import pandas as pd` | Essential for **data manipulation** and analysis. Used for working with **DataFrames** (tabular data structure). |
| `import time` | Provides various time-related functions. Often used to measure the **execution time (performance)** of code blocks, especially database operations. |
| `import sqlalchemy as db` | The main SQLAlchemy library. Used for accessing fundamental types and functions (though often redundant if specific modules are imported). |
| `from sqlalchemy import create_engine` | The key function to **establish a connection** to a database (e.g., SQLite, PostgreSQL, MySQL). |
| `from sqlalchemy import text` | Used to wrap **raw SQL strings** when executing them directly via SQLAlchemy, allowing for easier parameter binding and compatibility. |
| `from sqlalchemy_utils import create_database` | A helper function to easily **create a new database** if it doesn't already exist. |
| `from sqlalchemy_utils import database_exists` | A utility function to **check if a database** with a given connection URL **exists**. |
| `from typing import List` | Imports the List type, used to hint that a variable should be a list containing a specific type of elements (e.g., List[str]). |
| `from typing import Dict` | Imports the Dict type, used to hint that a variable should be a dictionary with specific key and value types (e.g., Dict[str, int]). |
| `import streamlit` | Used for interactive UI and dashboard pages, cache DB engine and display metrics, tables and charts |
| `import altair` | Used to visualisations integrated into Streamlit for line chart, bar chart and integrates with pandas DataFrame and Streamlit. |
| `import numpy` | Used for numeric transformations. |
| `import psycopg2` | Used as PostgreSQL database driver between Python applications and the PostgreSQL relational database. |
---

In [1]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi
import pandas as pd
import time
import sqlalchemy as db
from sqlalchemy import create_engine
from sqlalchemy_utils import create_database
from sqlalchemy_utils import database_exists
from sqlalchemy import text
from typing import List, Dict

---

## <font color='blue'>**Step 2** </font> - Retrieve resale_hdb Dataset from Kaggle

The dataset, sourced from **Kaggle** Dataset URL: https://www.kaggle.com/datasets/lzytim/hdb-resale-prices, contains Singapore Resale Flat Prices (Jan-17 to Sep-25).

---

In [2]:
# Create kaggle folder to store kaggle api token
#import os

if not os.path.exists('.kaggle'):
    os.makedirs('.kaggle')

In [3]:
import os
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi

def download_kaggle_dataset(dataset_name, download_path="datasets"):
    """
    Downloads a dataset from Kaggle using Kaggle API.

    :param dataset_name: Name of the Kaggle dataset (e.g. 'xyz/some-dataset-name').
    :param download_path: Path where to save the downloaded dataset.
    """
    
    # Make the download path if it doesn't exist (the 'datasets' and '.kaggle' folder)
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    if not os.path.exists('.kaggle'):
        os.makedirs('.kaggle')


    # Set up Kaggle API client
    api = KaggleApi()
    api.authenticate()
    
    # Download the dataset
    print(f"Downloading dataset {dataset_name}...")
    api.dataset_download_files(dataset_name, path=download_path, unzip=True)
    print(f"Dataset {dataset_name} downloaded to {download_path}")

def load_csv_from_dataset(download_path, csv_filename):
    """
    Loads a CSV file from a downloaded dataset.

    :param download_path: Path where the dataset was downloaded.
    :param csv_filename: Name of the CSV file to load.
    """
    # Construct the full path to the CSV file
    csv_file_path = os.path.join(download_path, csv_filename)

    # Load the CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)

    return df

# Example usage
dataset_name = 'lzytim/hdb-resale-prices'  # Replace this with the actual Kaggle dataset identifier
download_path = './datasets'  # Path where you want the dataset to be downloaded

# Download the dataset
download_kaggle_dataset(dataset_name, download_path)

# Detect .csv and add name of csv file to csv_filename
entries = os.listdir('datasets')
for n in entries:
    if '.csv' in n:
        csv_filename = n

# Load the CSV file
df = load_csv_from_dataset(download_path, csv_filename)

# Show first few rows of the DataFrame
print(df.head(1))


Dataset URL: https://www.kaggle.com/datasets/lzytim/hdb-resale-prices
Dataset lzytim/hdb-resale-prices downloaded to ./datasets
   Unnamed: 0       month        town blk_no             road_name building  \
0           0  2017-01-01  ANG MO KIO    406  ANG MO KIO AVENUE 10      NIL   

   postal  resale_price storey_range flat_type  ...             y  latitude  \
0  560406      232000.0     10 TO 12    2 ROOM  ...  38229.067463  1.362005   

   longitude  closest_mrt_station  distance_to_mrt_meters  transport_type  \
0  103.85388           Ang Mo Kio              999.941618             MRT   

   line_color distance_to_cbd         closest_pri_school  \
0         Red     8615.656983  TOWNSVILLE PRIMARY SCHOOL   

   distance_to_pri_school_meters  
0                     218.125254  

[1 rows x 30 columns]


---

## <font color='blue'>**Step 3** </font> - Retrieve MRT LRT Dataset from Kaggle

The dataset, sourced from **Kaggle** Dataset URL: https://www.kaggle.com/datasets/lzytim/hdb-resale-prices, contains Singapore Resale Flat Prices (Jan-17 to Sep-25).

---

In [4]:
# Example usage
dataset_train_name = "lzytim/full-list-of-mrt-and-lrt-stations-in-singapore"  # Replace this with the actual Kaggle dataset identifier
download_train_path = './train_datasets'  # Path where you want the dataset to be downloaded

# Download the dataset
download_kaggle_dataset(dataset_train_name, download_train_path)

# Detect .csv and add name of csv file to csv_filename
entries = os.listdir('train_datasets')
for n in entries:
    if '.csv' in n:
        csv_filename = n

# Load the CSV file
trains_df = load_csv_from_dataset(download_train_path, csv_filename)

# Show first few rows of the DataFrame

trains_df = trains_df[['station_name', 'type', 'color']]
trains_df.rename(columns={'station_name': 'station', 'type': 'transport_type', 'color': 'line_color'}, inplace=True)

print(trains_df.head())


Dataset URL: https://www.kaggle.com/datasets/lzytim/full-list-of-mrt-and-lrt-stations-in-singapore
Dataset lzytim/full-list-of-mrt-and-lrt-stations-in-singapore downloaded to ./train_datasets
         station transport_type line_color
0    Jurong East            MRT        Red
1    Bukit Batok            MRT        Red
2   Bukit Gombak            MRT        Red
3  Choa Chu Kang            MRT        Red
4        Yew Tee            MRT        Red



---

## <font color='blue'>**Step 4** </font> - Setup postgreSQL Connection details

* Define pg_config with postgres connection details.
* Define variables containing the Database name, tablenames that are to be created and used for this project.

---


In [5]:
# -------------------------------------------------------------
# Connection details / DB_Name / Table_name
# -------------------------------------------------------------
pg_config = {
    'host': 'localhost',
    'port': 5432,
    'user': 'postgres',
    'password': 'admin'
}

new_db_name = "hdb_db"
table_name = "resale_hdb"
town_tablename = 'towns'
trains_tablename = 'trains'
stations_tablename = 'stations'



---

## <font color='blue'>**Step 5** </font> - Create postgresql DB if not exists (hdb_db)

* Check if database name hdb_db already exists
* Create new DB with database name if not exists

---


In [6]:
def create_postgresql_database(pg_config: dict, new_db_name: str):
    # 1. Construct the connection URL
    # NOTE: We construct the URL without the DB name first, as the DB must exist to connect.
    # We then add the new_db_name to the URL object for the existence check.
    base_url = f"postgresql://{pg_config['user']}:{pg_config['password']}@{pg_config['host']}:{pg_config['port']}"
    
    # 2. Create the engine object
    # We create the engine pointing to the target database URL
    engine = create_engine(f"{base_url}/{new_db_name}")

    # 3. Check for existence and create
    try:
        if not database_exists(engine.url):
            create_database(engine.url)
            print(f"Database '{new_db_name}' created successfully.")
        else:
            print(f"Database '{new_db_name}' already exists.")
            
    except Exception as e:
        print(f"An error occurred during database creation: {e}")
        
    finally:
        # 4. Release resources associated with the engine
        engine.dispose()

In [7]:
# ----- Function - call ----- #
create_postgresql_database(pg_config, new_db_name)

Database 'hdb_db' already exists.



---

# <font color='blue'>**Step 6** </font> - Assign Kaggle dataframe to RAWDATA dataframe

* This assign the dataframe from Kaggle in step2 to dataframe rawdata.
* This assignment is for integration testing as we are working on the 2 parts individually.

---


In [8]:

#rawdata = pd.read_csv(csv_file)
rawdata = df
print("Kaggle resale hdb dataframe file loaded to rawdata dataframe:")

rawdata.head(1)

Kaggle resale hdb dataframe file loaded to rawdata dataframe:


,Unnamed: 0,month,town,blk_no,road_name,building,postal,resale_price,storey_range,flat_type,...,y,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters
0,0,2017-01-01,ANG MO KIO,406,ANG MO KIO AVENUE 10,NIL,560406,232000.0,10 TO 12,2 ROOM,...,38229.067463,1.362005,103.85388,Ang Mo Kio,999.941618,MRT,Red,8615.656983,TOWNSVILLE PRIMARY SCHOOL,218.125254



---

## <font color='blue'>**Step 7** </font> - Create TRAINS, STATIONS, TOWNS, HDB_RESALE tables in postgresql

* Drop all tables before recreation
* STATIONS - to contain all the station names
* TRAINS - to contain all the train type (MRT, LRT) and line color information.
* TOWNS - to contain town name, region_ura, planning_area_ura information (subset from resale hdb transactions file)
* RESALE_HDB - to contain all main information regarding HDB resale transactions

---


In [9]:
def execute_sql_commands(pg_config: Dict, db_name: str, sql_commands: List[str]):
    """
    Args:
        pg_config (Dict): Dictionary with PostgreSQL connection details 
                          ('user', 'password', 'host', 'port').
        db_name (str): The name of the target database where commands will run.
        sql_commands (List[str]): A list of SQL strings to be executed.
    """
    
    # 1. Construct the connection URL
    try:
        db_url = (
            f"postgresql://{pg_config['user']}:{pg_config['password']}@"
            f"{pg_config['host']}:{pg_config['port']}/{db_name}"
        )
        
        # 2. Create the engine
        engine = create_engine(db_url)

        # 3. Execute commands within a transaction
        # engine.begin() manages the transaction, commits on success, rolls back on error
        with engine.begin() as conn:
            print(f"Executing commands on database '{db_name}'...")
            for i, command in enumerate(sql_commands):
                # Using text() is best practice for executing raw SQL strings
                conn.execute(text(command))
                print(f"  > SQL Command {i+1}/{len(sql_commands)} executed.")
        
        print("All SQL commands executed successfully.")

    except Exception as e:
        print(f"An error occurred during SQL execution: {e}")
        
    finally:
        # 4. Release resources
        if 'engine' in locals() and engine:
            engine.dispose()

In [10]:
engine = create_engine(
    f"postgresql://{pg_config['user']}:{pg_config['password']}@{pg_config['host']}:{pg_config['port']}/{new_db_name}"
)

sql_commands = ['''
    DROP TABLE IF EXISTS stations CASCADE;
    CREATE TABLE stations(
        station_id SERIAL PRIMARY KEY,
        station VARCHAR UNIQUE NOT NULL
        );
    ''',
    '''
    DROP TABLE IF EXISTS trains CASCADE;
    CREATE TABLE trains(
        id SERIAL PRIMARY KEY ,
        station VARCHAR REFERENCES stations(station),
        transport_type VARCHAR,
        line_color VARCHAR,
        UNIQUE (station, transport_type, line_color)
    );
   ''',
   '''             
    DROP TABLE IF EXISTS towns CASCADE;
    CREATE TABLE towns(
        town_id INT PRIMARY KEY,
        town VARCHAR,
        planning_area_ura VARCHAR,
        region_ura VARCHAR
    );
   ''',
   '''
    DROP TABLE IF EXISTS resale_hdb CASCADE;  
    CREATE TABLE resale_hdb(   
        resale_id SERIAL PRIMARY KEY,
        month date,
        postal text,
        resale_price int,
        storey_range text,
        flat_type text,
        flat_model text,
        remaining_lease_years int,
        floor_area_sqft numeric,
        price_per_sqft numeric,
        latitude numeric,
        longitude numeric,
        closest_mrt_station text REFERENCES stations(station),
        distance_to_mrt_meters numeric,
        distance_to_cbd numeric,
        closest_pri_school text,
        distance_to_pri_school_meters numeric,
        town_id int REFERENCES towns(town_id)
    );
   ''',
   '''
   CREATE INDEX idx_trains_station ON trains(station);
   CREATE INDEX idx_resale_station ON resale_hdb(closest_mrt_station);
   '''
]

# ----- Function - call ----- #
execute_sql_commands(pg_config, new_db_name, sql_commands)

Executing commands on database 'hdb_db'...
  > SQL Command 1/5 executed.
  > SQL Command 2/5 executed.
  > SQL Command 3/5 executed.
  > SQL Command 4/5 executed.
  > SQL Command 5/5 executed.
All SQL commands executed successfully.



---

## <font color='blue'>**Step 8** </font> - Assign Kaggle MRT LRT dataset to Train dataframe

* load to trains dataframe

---


In [11]:
# distinct_train_df = pd.read_csv(trains_csv_file) -- assign from Kaggle dataset
distinct_train_df = trains_df
distinct_train_df.head(2)

,station,transport_type,line_color
0,Jurong East,MRT,Red
1,Bukit Batok,MRT,Red



---

## <font color='blue'>**Step 9** </font> - **Load** dataframe to Trains and Stations Table
* Stations.station_id is SERIAL type
* Trains.id is SERIAL type

info

---


In [12]:
# Define the columns to check, remove duplicates
subset_distinct_train_df = distinct_train_df[['station','transport_type','line_color']].drop_duplicates()

# Define the columns to check, remove duplicates
subset_distinct_stations_df = distinct_train_df['station'].drop_duplicates()

start = time.time()

subset_distinct_stations_df.to_sql(stations_tablename, engine, if_exists="append", index=False)
subset_distinct_train_df.to_sql(trains_tablename, engine, if_exists="append", index=False)

end = time.time()

print(f"Table '{trains_tablename}' and '{stations_tablename}' data loaded successfully!")
print(f"Runtime: {end - start:.4f} seconds")


Table 'trains' and 'stations' data loaded successfully!
Runtime: 0.2331 seconds



---

## <font color='blue'>**Step 10** </font> - Create subset TOWNS dataframe from RAWDATA dataframe
* Create Towns dataframe with a subset from original dataframe, rawdata
* Assigned town_id to the new town dataframe

---


In [13]:
distinct_town_df = rawdata[['town', 'planning_area_ura', 'region_ura']].drop_duplicates().sort_values('town')

distinct_town_df = distinct_town_df.reset_index(drop=True)
distinct_town_df.insert(0, "town_id", range(1, len(distinct_town_df) + 1))

distinct_town_df.head(2)

,town_id,town,planning_area_ura,region_ura
0,1,ANG MO KIO,ANG MO KIO,NORTH-EAST REGION
1,2,BEDOK,BEDOK,EAST REGION



---

## <font color='blue'>**Step 11** </font> - Update Town_id (from allocated TOWNS dataframe) to rawdata_with_town_id dataframe

* Reassign town_id back to original dataframe data
* Foriegn key on main table town_id referencing towns town_id to ensure data integrity 
---


In [14]:
rawdata_with_town_id = rawdata.merge(
    distinct_town_df[['town', 'planning_area_ura', 'region_ura', 'town_id']],
    on=['town', 'planning_area_ura', 'region_ura'],
    how='left'
)

rawdata_with_town_id.head(1)

,Unnamed: 0,month,town,blk_no,road_name,building,postal,resale_price,storey_range,flat_type,...,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,transport_type,line_color,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,town_id
0,0,2017-01-01,ANG MO KIO,406,ANG MO KIO AVENUE 10,NIL,560406,232000.0,10 TO 12,2 ROOM,...,1.362005,103.85388,Ang Mo Kio,999.941618,MRT,Red,8615.656983,TOWNSVILLE PRIMARY SCHOOL,218.125254,1



---

## <font color='blue'>**Step 12** </font> - **Load** Towns dataframe to Towns table

* loading dataframe to postgresql table

---


In [15]:
start = time.time()

distinct_town_df.to_sql(town_tablename, engine, if_exists="append", index=False)

end = time.time()

print(f"Table '{town_tablename}' created and CSV data loaded successfully!")
print(f"Runtime: {end - start:.4f} seconds")

Table 'towns' created and CSV data loaded successfully!
Runtime: 0.0128 seconds



---

## <font color='blue'>**Step 13** </font> - Data Cleaning - remove unnecessary data columns

* **Data fields to be excluded**
  * unnamed
  * 'blk_no', 
  * 'road_name', 
  * 'building', 
  * 'lease_commence_date',
  * 'remaining_lease_months',
  * 'floor_area_sqm',
  * 'x',
  * 'y',
  * 'town', 
  * 'planning_area_ura', 
  * 'region_ura', 
  * 'transport_type', 
  * 'line_color'

---


In [16]:
# -------------------------------------------------------------
# the first original column is "unnamed"
# This is to remove that column before write data to postgres database
# -------------------------------------------------------------

unnamed_cols = [col for col in rawdata_with_town_id.columns if 'Unnamed' in col]
drop_col_df = rawdata_with_town_id[unnamed_cols]
data_df = rawdata_with_town_id.drop(columns=unnamed_cols)

cols_to_exclude = ['blk_no', 
                   'road_name', 
                   'building', 
                   'lease_commence_date',
                   'remaining_lease_months',
                   'floor_area_sqm',
                   'x',
                   'y',
                   'town', 
                   'planning_area_ura', 
                   'region_ura', 
                   'transport_type', 
                   'line_color']
data_df = data_df.drop(columns=cols_to_exclude)

data_df.head(1)

,month,postal,resale_price,storey_range,flat_type,flat_model,remaining_lease_years,floor_area_sqft,price_per_sqft,latitude,longitude,closest_mrt_station,distance_to_mrt_meters,distance_to_cbd,closest_pri_school,distance_to_pri_school_meters,town_id
0,2017-01-01,560406,232000.0,10 TO 12,2 ROOM,Improved,61,473.6116,489.852867,1.362005,103.85388,Ang Mo Kio,999.941618,8615.656983,TOWNSVILLE PRIMARY SCHOOL,218.125254,1



---

## <font color='blue'>**Step 14** </font> - Data Cleaning - data formatting

* Convert Postal code to str type and ensure length of 6
* Convert to 2 decimal place
    * floor_area_sqft
    * price_per_sqft,
    * distance_to_mrt_meters
    * distance_to_cbd
    * distance_to_pri_school_meters
* Trim text variables
    *  closest_pri_school
    *  flat_model
* Check stations names matches stations list
  
---


In [17]:
# Convert Postal code to str type and ensure length of 6
data_df['postal'] = data_df['postal'].astype(str)
postal_mask = data_df['postal'].str.len() < 6

if len(data_df[postal_mask]) > 0:
    print(f"Cleaning {len(data_df[postal_mask])} postal details:")
    data_df['postal'] = data_df['postal'].str.zfill(6)
else:
    print("postal data checked ok ...")

# floor_area_sqft
data_df['floor_area_sqft'] = data_df['floor_area_sqft'].round(2)

# price_per_sqft  -- enhance next time to financial round up
data_df['price_per_sqft'] = data_df['price_per_sqft'].round(2)

#distance_to_mrt_meters (xx decimal place)
data_df['distance_to_mrt_meters'] = data_df['distance_to_mrt_meters'].round(2)

#distance_to_cbd (xx decimal place)
data_df['distance_to_cbd'] = data_df['distance_to_cbd'].round(2)

#distance_to_pri_school_meters (xx decimal place)
data_df['distance_to_pri_school_meters'] = data_df['distance_to_pri_school_meters'].round(2)

#TRIM text cleaning
#closest_pri_school(TRIM)
data_df['closest_pri_school'] = data_df['closest_pri_school'].str.strip()

#flat_model(Trim)
data_df['flat_model'] = data_df['flat_model'].str.strip()

# closest_mrt_stations (check if exists in trains df )
unclean_trains_df_mask = ~data_df['closest_mrt_station'].isin(subset_distinct_train_df['station'])
unclean_trains_records = data_df[unclean_trains_df_mask]

if len(unclean_trains_records) == 0:
    print("train stations names check okay ... ")
else:
    print(f"num of train stations names incorrect records {len(unclean_trains_records)}")
    print(unclean_trains_records.head())


Cleaning 2540 postal details:
num of train stations names incorrect records 4
             month  postal  resale_price storey_range flat_type  \
199094  2025-04-01  650265      400000.0     01 TO 03    3 ROOM   
199096  2025-05-01  650259      410000.0     04 TO 06    3 ROOM   
199100  2025-08-01  650265      430000.0     04 TO 06    3 ROOM   
199103  2025-09-01  650265      427000.0     07 TO 09    3 ROOM   

            flat_model  remaining_lease_years  floor_area_sqft  \
199094  New Generation                     59           721.18   
199096  New Generation                     59           721.18   
199100  New Generation                     59           882.64   
199103  New Generation                     59           882.64   

        price_per_sqft  latitude   longitude closest_mrt_station  \
199094          554.65  1.350008  103.759863                Hume   
199096          568.51  1.350859  103.759761                Hume   
199100          487.17  1.350008  103.759863       

---

## <font color='blue'>**Step 14.1** </font> - Data Cleaning - Due to Unclean Data in Step14.

MRT LRT dataset lastest updated on is Jan 2025
HDB Resale transactions Sep 2025
To clean the data - 
This portion is to insert the latest station 'Hume' station.

---

In [18]:
engine = create_engine(
    f"postgresql://{pg_config['user']}:{pg_config['password']}@{pg_config['host']}:{pg_config['port']}/{new_db_name}"
)

sql_commands = ['''
        INSERT INTO stations (station) VALUES ('Hume');
    ''',
    '''
        INSERT INTO trains (station, transport_type,
        line_color)
		VALUES ('Hume', 'MRT', 'Blue');
    '''
    ]

execute_sql_commands(pg_config, new_db_name, sql_commands)

Executing commands on database 'hdb_db'...
  > SQL Command 1/2 executed.
  > SQL Command 2/2 executed.
All SQL commands executed successfully.



---

## <font color='blue'>**Step 15** </font> - **Load** dataframe to resale_hdb Table

* Expected 217,001 records to be loaded 
* Expected loading time around 60-120 secs

---


In [19]:
# -------------------------------------------------------------
# Load CSV data to table (new table or replace)
# -------------------------------------------------------------
start = time.time()

data_df = data_df.reset_index(drop=True) 
data_df.index = data_df.index + 1        # shift to start at 1

#data_df.to_sql(table_name, engine, if_exists="append", index=True, index_label="resale_id") 
data_df.to_sql(table_name, engine, if_exists="append", index=False) 
end = time.time()

print(f"Table '{table_name}' data loaded successfully!")
print(f"Runtime: {end - start:.4f} seconds")

engine.dispose()

Table 'resale_hdb' data loaded successfully!
Runtime: 45.6591 seconds



---

## <font color='blue'>**Complete** </font>

* Below data verification of the count of tables
* Data can be accessed through postgresql database
* Accessible to any other tools for data analyzing i.e. tableau

---


---

## <font color='blue'>**Proceed next to streamlit app, Tableau data visualization** </font>

* Next program - Streamlit app will access the data from postgres database
* Tableau (next program) - access database for data analyzing data visualization

---

# Extra Example - Using sqlalchemy to query

In [ ]:
pd.read_sql("Select count(*) from resale_hdb", engine)

In [ ]:
pd.read_sql("select count(1) from towns", engine)

In [ ]:
pd.read_sql("select count(1) from trains", engine)

In [ ]:
pd.read_sql("select count(1) from stations", engine)